In [1]:
#################################
# filenames
#################################

number = 34

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(0.5)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(15)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(15)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    #time.sleep(0.5)
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(14)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(14)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(0.5)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(14)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(14)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(0.5)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo[:101])
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615346597.8196194
500
1/500 b7,4,57/6,4
histo1 1
2/500 664,36,542/3,3
histo1 1
3/500 b3,5,1,b7
histo1 1
4/500 4,47,4,642
histo1 1
5/500 5,2,27,242
histo1 1
6/500 1,5,1,56/6
histo1 2
7/500 b7,1,3,b7
histo1 1
8/500 164,47,4,1
histo1 1
9/500 6,3,4,6
histo1 14
10/500 1,564,6,3
histo1 1
11/500 4,1,57/6,4
histo1 1
12/500 4,57/6,57,7/5
histo1 1
13/500 b7,4,b7,264
histo1 1
14/500 56,464,164,56
histo1 1
15/500 6,5,6,27
histo1 3
16/500 67,M57,47,37
histo1 2
17/500 4,16,5,16
histo1 5
18/500 4,b7,4,1
histo1 8
19/500 2,6,664,66
histo1 1
20/500 57/4,665,464,1
histo1 1
21/500 5,3,57/2,5/2
histo1 1
22/500 4,b3,M3,4
histo1 1
23/500 2,66,464,564
histo1 1
24/500 5,6,3,47
histo1 3
25/500 364,4,5,1
histo1 5
26/500 57,37,17,57/6
histo1 1
27/500 2,664,26,7/5
histo1 1
28/500 27,b7,1,56
histo1 1
29/500 67,564,27,47
histo1 1
30/500 1,4,47,37
histo1 1
31/500 4,5,37,564
histo1 1
32/500 1,47,1,5
histo1 1
33/500 4,5,1,27
histo1 10
34/500 b6,D3,b6,D3
histo1 1
35/500 57/2,27,243,565
histo1 1
36/500 1,5,4,164
histo1 

histo1 1
291/500 6,17,47,5
histo1 1
292/500 17,47/4,17,47/4
2. Unable to extract for 17 47/4 17 47/4 page 1
histo2 20
293/500 464,564,6,464
histo1 1
294/500 5,4,D3,D76
histo1 1
295/500 66,464,1,4
histo1 1
296/500 464,1,16,442
histo1 1
297/500 b4,1,465,27
histo1 1
298/500 364,564,664,6
histo1 1
299/500 47,2,47/4,6
histo1 1
300/500 6,26,5,16
histo1 1
301/500 b4,1,2,264
histo1 1
302/500 164,5,2,464/4
histo1 1
303/500 1,542,16,542
histo1 1
304/500 3,6,L76,66
histo1 1
305/500 2,1,67,57
histo1 1
306/500 5,1,16,4
histo1 26
307/500 56,17,5,542
histo1 1
308/500 27,67,b57,27
histo1 1
309/500 2,37,57/2,2
histo1 1
310/500 464,M17,464,5
histo1 1
311/500 5/5,27,67,5/5
histo1 1
312/500 6,47,1,2
histo1 1
313/500 6,2,16,5
histo1 2
314/500 57/6,47,57/6,27
histo1 1
315/500 1,4,5,5/6
histo1 3
316/500 6,642,47,27
histo1 1
317/500 264,2,5,564
histo1 1
318/500 5,56/5,1,5
histo1 1
319/500 6,164,47,3
histo1 2
320/500 6,2,1,5/6
histo1 3
321/500 7,5,47,6
histo1 1
322/500 664,4,3,2
histo1 1
323/500 3,37,2,1
histo

In [ ]:
##########################
# list errors here
##########################
292/500 17,47/4,17,47/4
2. Unable to extract for 17 47/4 17 47/4 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

377/500 4,1,56,46
2. Unable to extract for 4 1 56 46 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [3]:
##########################
# test errors here
##########################
mykey = '4,1,56,46'
page = 1
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [4]:
# ##########################
# # fix histo here
# ##########################
# histo[20] -= 2
# print(histo[:21])
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 376, 52, 31, 9, 9, 5, 3, 2, 1, 3, 0, 1, 0, 1, 2, 1, 0, 0, 0, 0]
